# Sequence Final Project - Optimization

## First Approach to the Optimization Problem

### Requirements

In [ ]:
from pyomo.environ import *
import time
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import seaborn as sns

### Parameters

In [ ]:
def init_params(instance):
    with open(instance, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    qualifications={}
    i=0
    for q in sorted(data["qualifications"]):
        qualifications[q] = i
        i += 1   
    h= data["horizon"]
    qualif_nb = len(data["qualifications"])
    staff_nb = len(data["staff"])
    project_nb = len(data["jobs"])
    
    C = []
    for s in range(staff_nb):
        qualifs = data["staff"][s]["qualifications"]
        C.append([0]*qualif_nb)
        for q in qualifs:
            C[s][qualifications[q]]=1
    CA = [data["jobs"][j]["gain"] for j in range(len(data["jobs"]))]
    N = []
    for j in range(len(data["jobs"])):
        N.append([0]*qualif_nb)
        for q in data["jobs"][j]["working_days_per_qualification"].keys():
            N[j][qualifications[q]]=data["jobs"][j]["working_days_per_qualification"][q]
    G =[[1]*h for _  in range(staff_nb)]
    for s in range(staff_nb):
        for g in data["staff"][s]["vacations"]:
            G[s][g-1] = 0

    names = [staff["name"] for staff in data["staff"]]

    return(h,qualif_nb,staff_nb,project_nb,np.array(C),np.array(CA),np.array(N),np.array(G),names)

In [ ]:
# Décommenter la ligne que vous voulez utiliser pour l'instance
instance = "instances/toy_instance.json"
# instance = "instances/medium_instance.json" 
# instance = "instances/large_instance.json"
h,qualif_nb,staff_nb,project_nb,C,CA,N,G,names = init_params(instance)

### Solving function

In [ ]:
def solv_wf(h,staff_nb,qualif_nb,project_nb,CA,N,C,G):

    # --- 1. Model creation

    model = ConcreteModel()
    # Sets
    model.H = RangeSet(0,h-1)          # Planning Horizon
    model.S = RangeSet(0,staff_nb-1)        # Staff set
    model.P = RangeSet(0,project_nb-1)          # Projects set
    model.Q= RangeSet(0,qualif_nb-1)           # Qualifications set
        
    # Params
    
    model.h = Param(initialize=h)
    model.staff_nb = Param(initialize=staff_nb)
    model.qualif_nb = Param(initialize=qualif_nb)
    model.project_nb = Param(initialize=project_nb)

    # Variables
    
    model.w = Var(model.H, model.P, model.Q, model.S, within=Binary)
    model.y = Var(model.P,within=Binary)
    
    # Objective function
    
    def total_profit(model):
        profit = sum(
            model.y[p] * CA[p]
            for p in model.P)
        return profit

    model.obj = Objective(rule=total_profit, sense=maximize)
    
    # Constraints

    ## Constraint on the validation of the projects
    
    def validation_rule(model,p,q):
        return sum(model.w[t,p,q,s] for t in model.H for s in model.S) >= N[p,q] * model.y[p]
    model.validation = Constraint(model.P, model.Q, rule=validation_rule)

    ## Qualification constraint
    def qualification_rule(model,t,p,q,s):
        return  model.w[t, p, q, s] <= C[s, q]
    model.qualification = Constraint(model.H, model.P, model.Q, model.S, rule=qualification_rule)

    ## Day Off constraint
    def day_off_rule(model,t,s):
        return sum(model.w[t,p,q,s] for p in model.P for q in model.Q) <= G[s,t]
    model.day_off = Constraint(model.H, model.S, rule=day_off_rule)

    # --- 4. Solve
    
    # Nombre de variables
    num_var = len(list(model.component_data_objects(ctype=pyomo.environ.Var)))
    print("Nombre de variables :", num_var)

    # Nombre de paramètres
    num_param = len(list(model.component_data_objects(ctype=pyomo.environ.Param)))
    print("Nombre de paramètres :", num_param)

    # Nombre de contraintes
    num_constraint = len(list(model.component_data_objects(ctype=pyomo.environ.Constraint)))
    print("Nombre de contraintes :", num_constraint)
    
    start = time.time() 
    solver = SolverFactory("gurobi")  # or 'gurobi' if licensed
    results = solver.solve(model, tee=True)
    end = time.time()
    comp = end-start
    
    # -- 8. Transform it into computable forms
   
    w_arr = np.zeros((h,project_nb,qualif_nb,staff_nb))
    for t in range(h):
        for p in range(project_nb):
            for q in range(qualif_nb):
                for s in range(staff_nb):
                     w_arr[t, p, q, s] = value(model.w[t, p, q, s])

    y_arr = np.zeros((project_nb))
    
    for p in range(project_nb):
        y_arr[p] = value(model.y[p])
    
    return value(model.obj),w_arr,y_arr,comp,num_var,num_param,num_constraint
    

In [ ]:
obj,w_arr,y_arr,comp,num_var,num_param,num_constraint = solv_wf(h,staff_nb,qualif_nb,project_nb,CA,N,C,G)

## Visualisation of results

### Visualisation of planning

In [ ]:
table = pd.DataFrame(index=names, columns=range(h))

alp = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]

for t in range(h):
    for p in range(project_nb):
        for q in range(qualif_nb):
            for s in range(staff_nb):
                if w_arr[t, p, q, s] == 1:
                    table.iloc[s,t] = f"Job {p} {alp[q]}"

# table : DataFrame avec les valeurs "P X Y" ou NaN
table_plot = table.fillna("Day Off")

print("Optimization cost :", obj)
print("Computation time (s) :", comp)


In [ ]:
# Extraire uniquement les projets pour le mapping couleur
def extract_project(val):
    if val in ["", "Day Off"]:
        return "Day Off"
    return val.split()[1]  # "Proj X" → "X"

project_table = table_plot.applymap(extract_project)

# Liste des projets uniques (hors "Day Off")
unique_projects = sorted(list(set(v for v in project_table.values.flatten() if v not in ["", "Day Off"])))

# Mapping projet → entier pour heatmap
project_to_int = {"Day Off": 0}
project_to_int.update({v: i+1 for i, v in enumerate(unique_projects)})

int_table = project_table.replace(project_to_int).astype(int)

# Palette : blanc pour "Day Off", puis couleurs pour projets
palette = [(1,1,1)] + sns.color_palette("tab20", len(unique_projects))

# Plot
plt.figure(figsize=(h*1.5, staff_nb*1.5))
ax = sns.heatmap(int_table, annot=table_plot.values, fmt='',
                 cmap=palette,
                 cbar=False,
                 linewidths=0.5, linecolor='gray')

ax.set_xlabel("Time")
ax.set_ylabel("Staff")
ax.set_title("Staffing Schedule (Colored by project)")
plt.show()

### Optimization cost 

For the small instance, we obtain a cost of 70 and a computation time of 0.01s. 

For the medium instance, we obtain a cost of 440 and a computation time of 1.02s.

For the large instance, we obtain a cost of 860 and a computation time of 4.28s.